In [58]:
# Import necessary libraries
from sklearn.linear_model import *
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn.model_selection import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.pipeline import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
from sklearn.linear_model import *
import os

In [64]:
warnings.simplefilter("ignore")

# Change directory to the dataset location
os.chdir(r"D:\Datasets")

# Read the CSV data
df = pd.read_csv("Housing.csv")
df

,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
0,42000.0,5850,3,1,2,yes,no,yes,no,no,1,no
1,38500.0,4000,2,1,1,yes,no,no,no,no,0,no
2,49500.0,3060,3,1,1,yes,no,no,no,no,0,no
3,60500.0,6650,3,1,2,yes,yes,no,no,no,0,no
4,61000.0,6360,2,1,1,yes,no,no,no,no,0,no
...,...,...,...,...,...,...,...,...,...,...,...,...
541,91500.0,4800,3,2,4,yes,yes,no,no,yes,0,no
542,94000.0,6000,3,2,4,yes,no,no,no,yes,0,no
543,103000.0,6000,3,2,4,yes,yes,no,no,yes,1,no
544,105000.0,6000,3,2,2,yes,yes,no,no,yes,1,no


In [65]:

# Encode categorical features using one-hot encoding
dum_hous = pd.get_dummies(df, drop_first=True)

# Separate features (X) and target variable (y)
x = dum_hous.drop(['price'], axis=1)
y = dum_hous['price']
dum_hous  # (command to check version: pd.__version__ )  -> (latest version will give you (true,false))

,price,lotsize,bedrooms,bathrms,stories,garagepl,driveway_yes,recroom_yes,fullbase_yes,gashw_yes,airco_yes,prefarea_yes
0,42000.0,5850,3,1,2,1,True,False,True,False,False,False
1,38500.0,4000,2,1,1,0,True,False,False,False,False,False
2,49500.0,3060,3,1,1,0,True,False,False,False,False,False
3,60500.0,6650,3,1,2,0,True,True,False,False,False,False
4,61000.0,6360,2,1,1,0,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
541,91500.0,4800,3,2,4,0,True,True,False,False,True,False
542,94000.0,6000,3,2,4,0,True,False,False,False,True,False
543,103000.0,6000,3,2,4,1,True,True,False,False,True,False
544,105000.0,6000,3,2,2,1,True,True,False,False,True,False


In [61]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=24)

knn = KNeighborsRegressor()  # regressor
knn.fit(x_train,y_train)
y_pred = knn.predict(x_test)
print('r2 score(without scaling): ',r2_score(y_test,y_pred))

r2 score(without scaling):  0.3716245835208657


# standard scaling

In [62]:
std_scaler = StandardScaler()
pipe = Pipeline([('SCL',std_scaler),('KNN',knn)])
pipe.fit(x_train,y_train)
y_pred = pipe.predict(x_test)
print('r2 score(with scaling): ',r2_score(y_test,y_pred))

r2 score(with scaling):  0.64720082660368


# All in one

In [63]:
mn_scaler = MinMaxScaler()
std_scaler = StandardScaler()
knn = KNeighborsRegressor()
pipe = Pipeline([('SCL',mn_scaler),('KNN',knn)])
params= {'KNN__n_neighbors':np.arange(1,25),
         'SCL':[None,std_scaler,mn_scaler]} #(dunder)
kfold = KFold(n_splits=5,shuffle=True,random_state=24)
gcv = GridSearchCV(pipe,param_grid=params,cv =kfold)
gcv.fit(x,y)  #(gcv.fit actually a loop)

print(gcv.best_params_)
print(gcv.best_score_)

{'KNN__n_neighbors': 9, 'SCL': StandardScaler()}
0.6143509227778978
